In [16]:
import pandas as pd

In [17]:
df = pd.read_csv("secom.data", header=None,delimiter=" ")
df_y = pd.read_csv("secom_labels.data", header=None, delimiter=" ")
df_y.columns = ["label","TimeStamp"]

In [18]:
df_y["label"].value_counts()

label
-1    1463
 1     104
Name: count, dtype: int64

In [19]:
df['label'] = df_y['label']

In [20]:
print("Total rows: ", df.shape[0])
print("Total columns: ", df.shape[1])

Total rows:  1567
Total columns:  591


## Removing Columns with more that 10% missing values

In [21]:
columns_null = df.isnull().sum()
for i in range(len(columns_null)-1):
    if columns_null[i] > 0:
        print(f"Column {i} has {columns_null[i]} null values")

Column 0 has 6 null values
Column 1 has 7 null values
Column 2 has 14 null values
Column 3 has 14 null values
Column 4 has 14 null values
Column 5 has 14 null values
Column 6 has 14 null values
Column 7 has 9 null values
Column 8 has 2 null values
Column 9 has 2 null values
Column 10 has 2 null values
Column 11 has 2 null values
Column 12 has 2 null values
Column 13 has 3 null values
Column 14 has 3 null values
Column 15 has 3 null values
Column 16 has 3 null values
Column 17 has 3 null values
Column 18 has 3 null values
Column 19 has 10 null values
Column 21 has 2 null values
Column 22 has 2 null values
Column 23 has 2 null values
Column 24 has 2 null values
Column 25 has 2 null values
Column 26 has 2 null values
Column 27 has 2 null values
Column 28 has 2 null values
Column 29 has 2 null values
Column 30 has 2 null values
Column 31 has 2 null values
Column 32 has 1 null values
Column 33 has 1 null values
Column 34 has 1 null values
Column 35 has 1 null values
Column 36 has 1 null val

In [22]:
# remove column with more than 10% null values
df = df.dropna(thresh=len(df)*0.9, axis=1)

## Dropping columns with 0 standard deviation

In [23]:
# remove columns with zero standard deviation
df = df.loc[:, df.std() != 0]

## Filling missing values with mean or median based on distribution

In [24]:
normal_dis_columns = []
skewness_columns = []
for column in df.columns:
    skewness = df[column].skew()
    if abs(skewness) < 0.5:
        normal_dis_columns.append(column)
    else:
        skewness_columns.append(column)

In [25]:
# fill nan values with mean for normal distribution columns
for column in normal_dis_columns:
    df[column] = df[column].fillna(df[column].mean())
    
# fill nan values with median for skewness columns
for column in skewness_columns:
    df[column] = df[column].fillna(df[column].median())

## Correlation matrix
removal of highly correlated features

In [30]:
# find correlation between columns
correlation = df.corr()

# print columns with high correlation
high_correlation = []
for i in range(len(correlation.columns)):
    for j in range(i):
        if abs(correlation.iloc[i, j]) > 0.99:
            colname = correlation.columns[i]
            high_correlation.append((colname, correlation.columns[j]))
            
print(len(high_correlation), " columns with high correlation")

121  columns with high correlation


In [35]:
counts_corr = {}
for col1, col2 in high_correlation:
    if col1 not in counts_corr:
        counts_corr[col1] = 0
    if col2 not in counts_corr:
        counts_corr[col2] = 0
    counts_corr[col1] += 1
    counts_corr[col2] += 1
    
columns_with_high_corr_with_more_than_1_corr = []
for col, count in counts_corr.items():
    if count > 1:
        columns_with_high_corr_with_more_than_1_corr.append(col)

In [ ]:
removeal_columns = []
for col in columns_with_high_corr_with_more_than_1_corr:
    cols1 = [col2 for col1, col2 in high_correlation if col1 == col]
    cols2 = [col1 for col1, col2 in high_correlation if col2 == col]
    removeal_columns.extend(cols1)
    removeal_columns.extend(cols2)
    
removeal_columns = list(set(removeal_columns))

142  columns to remove
57  unique columns to remove


In [38]:
# remove columns with high correlation
for col in removeal_columns:
    if col in df.columns:
        df = df.drop(col, axis=1)

In [39]:
len(df.columns), " columns after removing high correlation columns"

(366, ' columns after removing high correlation columns')

In [42]:
df.to_csv("secom_cleaned.csv", index=False)